In [ ]:
import pandas as pd

# 1. 파일 로드 (파일명이 업로드하신 이름과 일치하는지 확인해주세요)
files = {
    2018: '2018 Overtakes - Sheet1.csv',
    2019: '2019 Overtakes - Sheet1.csv',
    2020: '2020 Overtakes - Sheet1.csv',
    '21_22': 'f1_detailed_overtakes_21_22_v2.csv',
    '23_24': 'f1_overtakes_2023_2024.csv'
}

# 드라이버 번호 -> 이름 매핑 (2018-2020 형식)
driver_map = {
    1: 'Verstappen', 33: 'Verstappen', 44: 'Hamilton', 16: 'Leclerc', 55: 'Sainz',
    4: 'Norris', 63: 'Russell', 11: 'Perez', 14: 'Alonso', 10: 'Gasly',
    31: 'Ocon', 22: 'Tsunoda', 23: 'Albon', 77: 'Bottas', 24: 'Zhou',
    27: 'Hulkenberg', 20: 'Magnussen', 18: 'Stroll', 81: 'Piastri', 2: 'Sargeant',
    3: 'Ricciardo', 47: 'Schumacher', 9: 'Mazepin', 6: 'Latifi', 7: 'Raikkonen',
    99: 'Giovinazzi', 5: 'Vettel', 88: 'Kubica', 26: 'Kvyat', 8: 'Grosjean',
    28: 'Hartley', 35: 'Sirotkin', 51: 'Fittipaldi', 89: 'Aitken', 21: 'De Vries',
    40: 'Lawson', 43: 'Colapinto', 38: 'Bearman'
}

# 레이스 명칭 통일 딕셔너리
race_fix = {'United States': 'USA', 'São Paulo': 'Brazil', 'Emilia Romagna': 'Italy', 'Abu Dhabi Grand Prix': 'Abu Dhabi'}

def clean_race(name):
    if not isinstance(name, str): return name
    n = name.replace(' Grand Prix', '').strip()
    return race_fix.get(n, n)

# 데이터 처리 및 병합
all_data = []

# 2018-2020 처리
for year in [2018, 2019, 2020]:
    df = pd.read_csv(files[year])
    df = df.rename(columns={'Race': 'race', 'Overtaker': 'overtaker', 'Position': 'position'})
    df['date'] = str(year)
    all_data.append(df[['race', 'overtaker', 'date', 'position']])

# 2021-2022 처리
df2122 = pd.read_csv(files['21_22'])
df2122['race'] = df2122['GrandPrix'].apply(clean_race)
df2122['overtaker'] = df2122['Driver'].map(driver_map).fillna(df2122['Driver'].astype(str))
df2122['date'] = df2122['Year'].astype(str)
df2122['position'] = df2122['PostPosition']
all_data.append(df2122[['race', 'overtaker', 'date', 'position']])

# 2023-2024 처리 (간소화된 매핑 적용)
df2324 = pd.read_csv(files['23_24'])
# 23-24 데이터는 이미 date가 있으므로 그대로 사용, 드라이버 번호만 변환
df2324['overtaker'] = df2324['overtaking_driver_number'].map(driver_map).fillna(df2324['overtaking_driver_number'].astype(str))
df2324 = df2324.rename(columns={'meeting_key': 'race'}) # 실제 분석시엔 meeting_key 매핑 권장
all_data.append(df2324[['race', 'overtaker', 'date', 'position']])

# 최종 병합 및 저장
final_merged = pd.concat(all_data, ignore_index=True)
final_merged['race'] = final_merged['race'].apply(clean_race)
final_merged.to_csv('f1_overtakes_merged_final.csv', index=False)
print("파일 생성 완료: f1_overtakes_merged_final.csv")

In [ ]:
import pandas as pd

# 1. 파일 설정 (파일명이 실제 파일과 일치하는지 확인하세요)
file_names = {
    '2018': '2018 Overtakes - Sheet1.csv',
    '2019': '2019 Overtakes - Sheet1.csv',
    '2020': '2020 Overtakes - Sheet1.csv',
    '21_22': 'f1_detailed_overtakes_21_22_v2.csv',
    '23_24': 'f1_overtakes_2023_2024.csv'
}

# 2. 드라이버 번호 -> 약자 이름 매핑
driver_map = {
    1: 'Verstappen', 33: 'Verstappen', 44: 'Hamilton', 16: 'Leclerc', 55: 'Sainz',
    4: 'Norris', 63: 'Russell', 11: 'Perez', 14: 'Alonso', 10: 'Gasly',
    31: 'Ocon', 22: 'Tsunoda', 23: 'Albon', 77: 'Bottas', 24: 'Zhou',
    27: 'Hulkenberg', 20: 'Magnussen', 18: 'Stroll', 81: 'Piastri', 2: 'Sargeant',
    3: 'Ricciardo', 47: 'Schumacher', 9: 'Mazepin', 6: 'Latifi', 7: 'Raikkonen',
    99: 'Giovinazzi', 5: 'Vettel', 88: 'Kubica', 26: 'Kvyat', 8: 'Grosjean',
    28: 'Hartley', 35: 'Sirotkin', 51: 'Fittipaldi', 89: 'Aitken', 21: 'De Vries',
    40: 'Lawson', 43: 'Colapinto', 38: 'Bearman'
}

# 3. 레이스 명칭 통일용 딕셔너리
race_fix = {
    'United States': 'USA', 'São Paulo': 'Brazil', 'Emilia Romagna': 'Italy', 
    'Emilia-Romagna': 'Italy', 'Portuguese': 'Portugal', 'Sakhir': 'Bahrain',
    '70th Anniversary': 'Britain', 'Tuscany': 'Italy', 'Styrian': 'Styria'
}

# 2023-2024년 장소 매핑 (날짜 기준)
calendar_map = {
    "2023-03-05": "Bahrain", "2023-03-19": "Saudi Arabia", "2023-04-02": "Australia",
    "2023-04-30": "Azerbaijan", "2023-05-07": "Miami", "2023-05-28": "Monaco",
    "2023-06-04": "Spain", "2023-06-18": "Canada", "2023-07-02": "Austria",
    "2023-07-09": "Britain", "2023-07-23": "Hungary", "2023-07-30": "Belgium",
    "2023-08-27": "Netherlands", "2023-09-03": "Italy", "2023-09-17": "Singapore",
    "2023-09-24": "Japan", "2023-10-08": "Qatar", "2023-10-22": "USA",
    "2023-10-29": "Mexico", "2023-11-05": "Brazil", "2023-11-18": "Las Vegas",
    "2023-11-26": "Abu Dhabi", "2024-03-02": "Bahrain", "2024-03-09": "Saudi Arabia",
    "2024-03-24": "Australia", "2024-04-07": "Japan", "2024-04-21": "China"
}

def get_race_name(row):
    # 2023-2024용: 날짜에서 YYYY-MM-DD 추출 후 매핑
    d_str = str(row['date'])[:10]
    return calendar_map.get(d_str, "Unknown")

# 4. 데이터 처리 및 병합
all_dfs = []

# 2018-2020
for year in ['2018', '2019', '2020']:
    df = pd.read_csv(file_names[year])
    df = df.rename(columns={'Race': 'race', 'Overtaker': 'overtaker', 'Overtakee': 'overtakee', 'Position': 'position'})
    df['date'] = year
    # 2018년 드라이버 #9는 Ericsson으로 특수 처리
    if year == '2018':
        df.loc[df['overtaker'] == '9', 'overtaker'] = 'Ericsson'
    all_dfs.append(df[['race', 'overtaker', 'overtakee', 'date', 'position']])

# 2021-2022 (Overtakee가 없는 데이터)
df2122 = pd.read_csv(file_names['21_22'])
df2122['race'] = df2122['GrandPrix'].str.replace(' Grand Prix', '').apply(lambda x: race_fix.get(x, x))
df2122['overtaker'] = df2122['Driver'].map(driver_map)
df2122['overtakee'] = 'Unknown'
df2122['date'] = df2122['Year'].astype(str)
df2122['position'] = df2122['PostPosition']
all_dfs.append(df2122[['race', 'overtaker', 'overtakee', 'date', 'position']])

# 2023-2024
df2324 = pd.read_csv(file_names['23_24'])
df2324['race'] = df2324.apply(get_race_name, axis=1)
df2324['overtaker'] = df2324['overtaking_driver_number'].map(driver_map)
df2324['overtakee'] = df2324['overtaken_driver_number'].map(driver_map)
all_dfs.append(df2324[['race', 'overtaker', 'overtakee', 'date', 'position']])

# 5. 최종 병합 및 저장
final_df = pd.concat(all_dfs, ignore_index=True)
# 최종 레이스 명칭 한번 더 정리
final_df['race'] = final_df['race'].apply(lambda x: race_fix.get(x, x))

final_df.to_csv('f1_overtakes_2018_2024_full.csv', index=False)
print("성공! 'f1_overtakes_2018_2024_full.csv' 파일이 생성되었습니다.")

In [ ]:
import pandas as pd

# 1. 데이터 로드
df = pd.read_csv('f1_overtakes_2018_2024_full.csv')

# 2. 트랙 가중치(TrackWeight) 계산
# 각 레이스(race)와 날짜(date)별 추월 횟수를 구함
race_counts = df.groupby(['race', 'date']).size().reset_index(name='count')

# 트랙별 평균 추월 횟수 계산
track_avg = race_counts.groupby('race')['count'].mean()

# 추월이 적을수록 높은 점수를 주도록 역수 취함 (가중치)
# 전체 평균 가중치를 1로 맞추기 위해 (전체평균 / 트랙평균) 사용
global_avg = track_avg.mean()
track_weight = global_avg / track_avg

# 3. 개별 추월 점수 계산
# PositionScore: (21 - position)
# TrackWeight: 위에서 구한 트랙별 가중치 매핑
df['pos_score'] = 21 - df['position']
df['track_weight'] = df['race'].map(track_weight)

# 최종 점수 = 위치 점수 * 트랙 가중치
df['final_score'] = df['pos_score'] * df['track_weight']

# 4. 선수별(overtaker) 총점 합계 및 순위
driver_ranking = df.groupby('overtaker')['final_score'].sum().sort_values(ascending=False).reset_index()

print("--- F1 드라이버 추월 능력 평가 순위 (2018-2024) ---")
print(driver_ranking.head(10))